### Core capabilities of LLMs:

1. Reasoning (Thinking): LLMs understand questions, break them down, and figure out how to answer them.

2. Language Generation (Speaking): Once understood, LLMs generate word-by-word responses, effectively “speaking” back to users.

### Limitations of LLMs:
Despite their reasoning and language generation abilities, LLMs cannot perform real-world tasks such as booking train tickets, running code, interacting with APIs, or fetching live data (e.g., weather updates).
This is likened to a human body with a brain (thinking and speaking) but no hands or legs (cannot execute tasks physically).
## What is Tools?
A tool is a Python function that is packaged so that the LLM can call it when needed. The LLM decides which tool to use and provides inputs; the tool executes and returns results to the LLM.

### Types of Tools in LangChain

#### Built-in Tools:
These are pre-built, production-ready tools integrated into LangChain to cover common use cases such as Google searches, Wikipedia queries, running Python code, command-line operations, HTTP requests, sending Gmail messages, Slack integrations, database queries, and more. These tools require minimal setup and no coding from users.

#### Custom Tools:
When your use case is unique or specific to your application (e.g., a booking system for your company), you might need to build your own tools. Custom tools allow you to encapsulate your business logic or connect to your application’s APIs and databases directly.

### Built in tools

In [1]:
from langchain_community.tools import DuckDuckGoSearchRun

search_tool = DuckDuckGoSearchRun()

results = search_tool.invoke('top news in india today')

print(results)

t:\LangGraph\Langchain\.venv\Lib\site-packages\langchain_community\utilities\duckduckgo_search.py:63: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  with DDGS() as ddgs:


News in India: Check today's news brief on Politics, Sports, Business, Education & Entertainment news on Times of India Aug 18, 2025 · Top News Stories of the day, Latest News Headlines, News Specials, Breaking News and Latest India News, World current … India News, Latest News India: Read Live Breaking News from India along with Latest News and Today's Top Headlines Online in … Top News Headlines Today: Find current Top Stories in India from India Today. Top news stories from world, politics, business, sports, … Jul 13, 2025 · Stay informed with breaking news and latest news on politics, business, entertainment sports, science, technology …


In [7]:
from langchain_community.tools import ShellTool

shell_tool = ShellTool()
results = shell_tool.invoke("whoami")


print(results)

Executing command:
 whoami
desktop-ritvsgj\acer



t:\LangGraph\Langchain\.venv\Lib\site-packages\langchain_community\tools\shell\tool.py:33: UserWarning: The shell tool has no safeguards by default. Use at your own risk.
  warnings.warn(


### Custom tools are necessary when:

- You need to call your own APIs (e.g., for a travel booking app).

- You want to encapsulate unique business logic that is not covered by existing tools.

- You want your LLM to interact with your own databases or applications.

In [8]:
from langchain_core.tools import tool

In [ ]:
@tool # Add the Decoration
def multiply(a:int, b:int) -> int:
    """ Multiply two numbers """
    return a * b

@tool
def add(a:int, b:int) -> int:
    """ Add two numbers """
    return a + b

results = multiply.invoke({"a": 2, "b": 3})

In [10]:
results

6

In [12]:
print(multiply.name)
print(multiply.description)
print(multiply.args)


multiply
Multiply two numbers
{'a': {'title': 'A', 'type': 'integer'}, 'b': {'title': 'B', 'type': 'integer'}}


In [ ]:
# When you invoke the tool, you can pass the arguments as a dictionary to llm 
print(multiply.args_schema.model_json_schema())

{'description': 'Multiply two numbers ', 'properties': {'a': {'title': 'A', 'type': 'integer'}, 'b': {'title': 'B', 'type': 'integer'}}, 'required': ['a', 'b'], 'title': 'multiply', 'type': 'object'}


##  Method 2 Structured Tools
In LangChain, a Structured Tool is a type of tool designed to interact with Large Language Models (LLMs) in a more controlled and reliable manner by enforcing a predefined input schema. Unlike unstructured tools that might accept free-form string inputs, structured tools require inputs that conform to a specific data structure, often defined using Pydantic models in Python.

In [14]:
from langchain.tools import StructuredTool
from pydantic import BaseModel,Field

class MultiplyInput(BaseModel):
    a: int = Field(required=True, description="The first number to multiply")
    b: int = Field(required=True, description="The second number to multiply")

class AddInput(BaseModel):
    a: int = Field(required=True, description="The first number to add")
    b: int = Field(required=True, description="The second number to add")




In [16]:
def multiply(a:int, b:int) -> int:
    """ Multiply two numbers """
    return a * b

def add(a:int, b:int) -> int:
    """ Add two numbers """
    return a + b

In [17]:
multiply_tool = StructuredTool.from_function(func=multiply,name="Multiply",description="Multiplies two numbers", input_schema=MultiplyInput)
add_tool = StructuredTool.from_function(func=add, name="Add", description="Adds two numbers", input_schema=AddInput)


In [18]:
result_multiply = multiply_tool.invoke({"a": 2, "b": 3})
result_add = add_tool.invoke({"a": 2, "b": 3})

In [19]:
print(result_multiply)
print(result_add)

6
5


## BaseTools 
BaseTool is an abstract class (a blueprint) that all tools in LangChain inherit from.

It defines how tools are structured, described, and executed, so that agents can interact with them in a predictable way.

Without this standardization, an LLM agent wouldn’t know how to call external functions consistently.

In [20]:
from langchain.tools import BaseTool
from typing import Type

class MultiplyInput(BaseModel):
    a: int = Field(required=True, description="The first number to add")
    b: int = Field(required=True, description="The second number to add")

In [21]:
class MultiplyTool(BaseTool):
    name: str = "multiply"
    description: str = "Multiply two numbers"

    args_schema: Type[BaseModel] = MultiplyInput

    def _run(self, a: int, b: int) -> int:
        return a * b

In [ ]:
result = multiply_tool.invoke({'a':3, 'b':3})

print(result)
print(multiply_tool.name)
print(multiply_tool.description)

print(multiply_tool.args)

9
Multiply
Multiplies two numbers
{'a': {'title': 'A', 'type': 'integer'}, 'b': {'title': 'B', 'type': 'integer'}}


: 

### Toolkits in LangChain
A Toolkit is a collection of related tools packaged together for convenience and reusability.

In [1]:
from langchain_core.tools import tool

# Custom tools
@tool
def add(a: int, b: int) -> int:
    """Add two numbers"""
    return a + b

@tool
def multiply(a: int, b: int) -> int:
    """Multiply two numbers"""
    return a * b


In [2]:
class MathToolkit:
    def get_tools(self):
        return [add, multiply]

In [3]:
toolkit = MathToolkit()
tools = toolkit.get_tools()

for tool in tools:
    print(tool.name, "=>", tool.description)

add => Add two numbers
multiply => Multiply two numbers


### Tool Binding 
 tool binding, which is the first technical step in integrating tools with LLMs. Tool binding involves:

Registering tools with the LLM so that it knows which tools are available.
Providing each tool’s name, description, and expected input format (input schema) to the LLM.
Enabling the LLM to understand how to invoke the tool correctly.

In [4]:
import os 
from dotenv import load_dotenv
from langchain_groq import ChatGroq
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage
import requests


In [11]:
load_dotenv()
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")
llm = ChatGroq(model="deepseek-r1-distill-llama-70b")
llm.invoke("Hello, how are you?")

AIMessage(content="<think>\n\n</think>\n\nHello! I'm just a virtual assistant, so I don't have feelings, but I'm here and ready to help you with whatever you need. How are you doing? 😊", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 42, 'prompt_tokens': 9, 'total_tokens': 51, 'completion_time': 0.169337885, 'prompt_time': 0.009902427, 'queue_time': 0.045511903, 'total_time': 0.179240312}, 'model_name': 'deepseek-r1-distill-llama-70b', 'system_fingerprint': 'fp_76307ac09b', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--2b3f3e86-e25e-492b-9236-792f1a462824-0', usage_metadata={'input_tokens': 9, 'output_tokens': 42, 'total_tokens': 51})

In [7]:
# tool create

@tool
def multiply(a: int, b: int) -> int:
  """Given 2 numbers a and b this tool returns their product"""
  return a * b
print(multiply.invoke({'a':3, 'b':4}))

12


In [12]:
llm_with_tools = llm.bind_tools([multiply])

### Tool Calling:
 When and How LLMs Use Tools
Once tools are bound, the next concept is tool calling, where the LLM decides during a conversation whether it should use a specific tool to answer a query. Tool calling means:

The LLM detects that a question requires a tool’s functionality.
It generates a structured output indicating the tool’s name and input arguments needed to invoke the tool.

In [13]:
llm_with_tools.invoke("can you multiply 3 and 4?")

AIMessage(content='', additional_kwargs={'reasoning_content': "Okay, the user is asking me to multiply 3 and 4. I need to use the multiply function provided. Let me see, the function takes two integers, a and b. So I should set a to 3 and b to 4. That should give me 12. Let me format the tool call correctly with the function name and the arguments in JSON. I think that's all I need to do here.\n", 'tool_calls': [{'id': '9tr0hpvnq', 'function': {'arguments': '{"a":3,"b":4}', 'name': 'multiply'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 119, 'prompt_tokens': 145, 'total_tokens': 264, 'completion_time': 0.522578974, 'prompt_time': 0.022225307, 'queue_time': 0.049615803, 'total_time': 0.544804281}, 'model_name': 'deepseek-r1-distill-llama-70b', 'system_fingerprint': 'fp_1bbe7845ec', 'service_tier': 'on_demand', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--12e77e62-9b18-4960-a223-b23964ebb8b4-0', tool_calls=[{'name': 'multiply', 'args': {'

Tool Execution: Running the Tool and Returning Results
The next step is tool execution, where the developer/program calls the tool function with the inputs suggested by the LLM during tool calling. The tool runs, produces an output, and this output is packaged as a tool message.

In [14]:
result = llm_with_tools.invoke(
"Can you multiply 3 and 4?"
)

In [15]:
tool_result = multiply.invoke(result.tool_calls[0])

In [16]:
tool_result

ToolMessage(content='12', name='multiply', tool_call_id='xmjpsw0sv')

In [17]:
# tool create
from langchain_core.tools import InjectedToolArg
from typing import Annotated

@tool
def get_conversion_factor(base_currency: str, target_currency: str) -> float:
  """
  This function fetches the currency conversion factor between a given base currency and a target currency
  """
  url = f'https://v6.exchangerate-api.com/v6/c754eab14ffab33112e380ca/pair/{base_currency}/{target_currency}'

  response = requests.get(url)

  return response.json()

@tool
def convert(base_currency_value: int, conversion_rate: Annotated[float, InjectedToolArg]) -> float:
  """
  given a currency conversion rate this function calculates the target currency value from a given base currency value
  """

  return base_currency_value * conversion_rate

In [18]:
get_conversion_factor.invoke({'base_currency':'USD','target_currency':'INR'})

{'result': 'success',
 'documentation': 'https://www.exchangerate-api.com/docs',
 'terms_of_use': 'https://www.exchangerate-api.com/terms',
 'time_last_update_unix': 1756166401,
 'time_last_update_utc': 'Tue, 26 Aug 2025 00:00:01 +0000',
 'time_next_update_unix': 1756252801,
 'time_next_update_utc': 'Wed, 27 Aug 2025 00:00:01 +0000',
 'base_code': 'USD',
 'target_code': 'INR',
 'conversion_rate': 87.6051}

In [19]:
convert.invoke({'base_currency_value':10, 'conversion_rate':85.16})

851.5999999999999

In [20]:
llm = ChatGroq(model="deepseek-r1-distill-llama-70b")
llm_with_tools = llm.bind_tools([get_conversion_factor, convert])


In [21]:
messages = [HumanMessage('What is the conversion factor between INR and USD, and based on that can you convert 10 inr to usd')]

In [22]:
ai_message = llm_with_tools.invoke(messages)

In [23]:
messages.append(ai_message)

In [24]:
import json

for tool_call in ai_message.tool_calls:
  # execute the 1st tool and get the value of conversion rate
  if tool_call['name'] == 'get_conversion_factor':
    tool_message1 = get_conversion_factor.invoke(tool_call)
    # fetch this conversion rate
    conversion_rate = json.loads(tool_message1.content)['conversion_rate']
    # append this tool message to messages list
    messages.append(tool_message1)
  # execute the 2nd tool using the conversion rate from tool 1
  if tool_call['name'] == 'convert':
    # fetch the current arg
    tool_call['args']['conversion_rate'] = conversion_rate
    tool_message2 = convert.invoke(tool_call)
    messages.append(tool_message2)

In [25]:
llm_with_tools.invoke(messages).content

''

In [27]:
from langchain.agents import initialize_agent, AgentType

# Step 5: Initialize the Agent ---
agent_executor = initialize_agent(
    tools=[get_conversion_factor, convert],
    llm=llm,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,  # using ReAct pattern
    verbose=True  # shows internal thinking
)

C:\Users\Acer\AppData\Local\Temp\ipykernel_28180\627313024.py:4: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent_executor = initialize_agent(


In [28]:
# --- Step 6: Run the Agent ---
user_query = "Hi how are you?"

response = agent_executor.invoke({"input": user_query})



> Entering new AgentExecutor chain...
<think>
Okay, the user greeted me with "Hi how are you?" I should respond politely. Since I'm an AI, I don't have feelings, but I can express that I'm here to help. I don't need any tools for this response because it's a straightforward greeting. I'll just say I'm doing well and offer my assistance.
</think>

Hello! I'm just a computer program, so I don't have feelings, but I'm here and ready to help you with whatever you need. How can I assist you today?

Action: ```{ "action": "Final Answer", "action_input": "Hello! I'm just a computer program, so I don't have feelings, but I'm here and ready to help you with whatever you need. How can I assist you today?" }```

> Finished chain.
